In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
!pip install gurobipy
import subprocess
import sys


import numpy as np
from gurobipy import *


def GurobiIntSolve(A,b,c):
	c = -c # Gurobi default is maximization
	varrange = range(c.size)
	crange = range(b.size)
	m = Model('LP')
	m.params.OutputFlag = 0 #suppres output
	X = m.addVars(varrange, lb=0.0, ub=GRB.INFINITY, vtype=GRB.INTEGER,
                 obj=c,
                 name="X")
	C = m.addConstrs((sum(A[i,j]*X[j] for j in varrange)==b[i] for i in crange),'C')
	m.params.Method = -1 # primal simplex Method = 0
	m.optimize()
	# obtain results
	solution = []; 
	for i in X:
		solution.append(X[i].X);
	solution = np.asarray(solution)
	return m.ObjVal,solution


def GurobiSolve(A,b,c,Method=0):
	#print('solving starts')
	c = -c # Gurobi default is maximization
	varrange = range(c.size)
	crange = range(b.size)
	m = Model('LP')
	m.params.OutputFlag = 0 #suppress output
	X = m.addVars(varrange, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS,
                 obj=c,
                 name="X")
	C = m.addConstrs((sum(A[i,j]*X[j] for j in varrange)==b[i] for i in crange),'C')
	m.params.Method = Method # primal simplex Method = 0
	#print('start optimizing...')
	m.optimize()
	# obtain results
	solution = []; basis_index = []; RC = []
	for i in X:
		solution.append(X[i].X);
		RC.append(X[i].getAttr('RC'))
		if X[i].getAttr('VBasis') == 0:
			basis_index.append(i)
	solution = np.asarray(solution)
	RC = np.asarray(RC)
	basis_index = np.asarray(basis_index)
	#print('solving completes')
	return m.ObjVal,solution,basis_index,RC


def computeoptimaltab(A,b,RC,obj,basis_index):
	'''
	A - A matrix, b - constraint, RC - reduced cost, basis_index - basis 
	'''
	m,n = A.shape
	assert m == b.size; assert n == RC.size
	B = A[:,basis_index]
	try:
		INV = np.linalg.inv(B)
	except:
		print('basisindex length:', basis_index.size)
		print('Ashape:', A.shape)
		raise ValueError
	x = np.dot(INV,b)
	A_ = np.dot(INV,A)
	firstrow = np.append(-obj,RC)
	secondrow = np.column_stack((x,A_))
	tab = np.vstack((firstrow,secondrow))
	return tab


def generatecutzeroth(row):
	###
	# generate cut that includes cost/obj row as well
	###
	n = row.size
	a = row[1:n]
	b = row[0]
	cut_a = a - np.floor(a)
	cut_b = b - np.floor(b)
	return cut_a,cut_b


def updatetab(tab,cut_a,cut_b,basis_index):
	cut_a = -cut_a
	cut_b = -cut_b
	m,n = tab.shape
	A_ = tab[1:m,1:n]; b_ = tab[1:m,0]; c_ = tab[0,1:n]; obj = tab[0,0]
	Anew1 = np.column_stack((A_,np.zeros(m-1)))
	Anew2 = np.append(cut_a,1)
	Anew = np.vstack((Anew1,Anew2))
	bnew = np.append(b_,cut_b)
	cnew = np.append(c_,0)
	M1 = np.append(obj,cnew)
	M2 = np.column_stack((bnew,Anew))
	newtab = np.vstack((M1,M2))
	basis_index = np.append(basis_index,n-1)
	return newtab,basis_index,Anew,bnew

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np


def phase_two(x,basis_index,A,b,c):
	'''
	executes phase_two of generic simplex algorithm
	x: starting BFS
	basis_index: index of variables that are basic feasible in x
	A: constraint matrix
	b: constraint
	c: cost
	'''
	# check consistency of basis_index and x
	for i in range(len(x)):
		if i not in basis_index:
			if abs(x[i])>1e-8:
				sys.exit('inconsistent input')
	# compute initial tab
	(m,n) = np.shape(A)
	B = A[:,basis_index]
	cB = c[basis_index]
	invB = np.linalg.inv(B)
	M1 = np.dot(invB,A)
	M2 = np.dot(invB,b)
	M3 = c - np.dot(cB,M1)
	M4 = -np.dot(cB,M2)
	A_b = np.column_stack((M1,M2))
	c_o = np.append(M3,M4)
	tab = np.vstack((A_b,c_o))
	counter = 1
	var_selection_order = []
	# iteration
	while np.sum(tab[m,0:n]<0)>1e-8:
		# find column with smallest index
		jset = np.where(tab[m,0:n]<-0)
		j = jset[0][0]
		# print out tab for error checking
		# print 'iteration',counter 
		# print 'tab',tab
		# check u<0
		if np.sum(tab[0:m,j]>1e-8)<1:
			bounded = 0
			return
		# find u>0 and compute ratio for comparison
		# bland's rule
		u = tab[0:m,j]
		xB = x[basis_index]
		ratio = np.zeros_like(xB)
		for i in range(len(xB)):
			if u[i]>1e-10:
				ratio[i] = xB[i]/(u[i]+0.0)
			else:
				ratio[i] = -10
		lset = np.where(ratio==np.min(ratio[u>1e-10]))
		l = lset[0][0]
		# change basis
		for i in range(m+1):
			if i!=l:
				tab[i,:] = tab[i,:] + (-tab[i,j]/(tab[l,j]+0.0)) * tab[l,:]
				tab[i,j] = 0 # for numerical stability #
		tab[l,:] = tab[l,:]/(tab[l,j]+0.0)
		# update basis index
		basis_index[l] = j
		# update solution
		x = np.zeros(n)
		X = tab[0:m,n]
		for i in range(len(basis_index)):
			idxset = np.where(tab[0:m,basis_index[i]]==np.max(tab[0:m,basis_index[i]]))
			idx = idxset[0]
			if len(idx)>1:
				print('tab')
				print(tab)
				print(tab[0:m,basis_index[i]])
				print('basis',basis_index[i])
				print('idx',idx)
				print('error of multiple indices')
				sys.exit()
			idx = idx[0]
			x[basis_index[i]] = X[idx]
		counter += 1
		var_selection_order.append(l)
		if counter > 10000:
			print(counter)
			print(var_selection_order)
			sys.exit('iteration exceeds maxiter')
	bounded = 1
	## change the layout
	m,n = tab.shape
	A_ = tab[0:m-1,0:n-1]
	b_ = tab[0:m-1,n-1]
	cbar = tab[m-1,0:n-1]
	obj = tab[m-1,n-1]
	M1 = np.append(obj,cbar)
	M2 = np.column_stack((b_,A_))
	tab = np.vstack((M1,M2))
	return tab,bounded,basis_index


def phase_one(A,b,c):
	'''
	executes phase one of generic simplex algorithm
	A: constraint matrix
	b: constraint
	c: cost
	'''
	# modify the input
	m,n = np.shape(A)
	for i in range(m):
		if b[i]<-1e-10:
			A[i,:] = -A[i,:]
			b[i] = -b[i]
	# solve an aux problem
	A_aux = np.concatenate((A,np.eye(m)),axis=1)
	b_aux = b
	c_aux = np.append(np.zeros(n),np.ones(m))
	x = np.append(np.zeros(n),b)
	basis_index = np.arange(n,n+m)
	tab,bounded,basis_index = phase_two(x,basis_index,A_aux,b_aux,c_aux)
	## change the layout
	m1,n1 = tab.shape
	A_ = tab[1:m1,1:n1]
	b_ = tab[1:m1,0]
	cbar = tab[0,1:n1]
	obj = tab[0,0]
	M1 = np.append(cbar,obj)
	M2 = np.column_stack((A_,b_))
	tab = np.vstack((M2,M1))
	# check feasibility
	if abs(tab[m,n+m]) < 1e-8:
		feasible = 1
	else:
		feasible = 0
		basis_index = []
		tab = []
		return x,basis_index,feasible,tab,A,b
 	# if feasible, drive artificial var out of the set
	while np.sum(basis_index>n-1)>0:
		lset = np.where(basis_index>n-1)
		l = lset[0][0]
		# if all are zeros, eliminte the redundant row
		if np.sum(abs(tab[l,0:n])) < 1e-5:
			index = []
			for ii in range(m):
				if ii!=l:
					index.append(ii)
			basis_index = basis_index[np.asarray(index)]
			A = A[np.asarray(index),:]
			b = b[np.asarray(index)]
			index = np.append(index,m)
			tab = tab[np.asarray(index),:]
		else: # can find a nonzero element
			jset = np.where(abs(tab[l,0:n])>1e-5)
			j = jset[0][0]
			m,n = A.shape
			for k in range(m):
				if k!=l:
					tab[k,:] = tab[k,:] + (-tab[k,j]/(0.0+tab[l,j])) * tab[l,:]
					tab[k,j] = 0 # for numerical stability #
			tab[l,:] = tab[l,:]/(0.0+tab[l,j])
			basis_index[l] = j
	m = len(basis_index)
	x = np.zeros(n)
	X = tab[0:m,n+m]
	for i in range(m):
		idxset = np.where(tab[0:m,basis_index[i]]>0.5)
		idxset = idxset[0]
		if len(idxset)>1:
			print('idx set size exceeds 1')
			sys.exit()
		else:
			x[basis_index[i]] = X[idxset[0]]
	## change the layout
	m,n = tab.shape
	A_ = tab[0:m-1,0:n-m]
	b_ = tab[0:m-1,n-1]
	cbar = tab[m-1,0:n-m]
	obj = tab[m-1,n-1]
	M1 = np.append(obj,cbar)
	M2 = np.column_stack((b_,A_))
	tab = np.vstack((M1,M2))
	A = A_
	b = b_
	return x,basis_index,feasible,tab,A,b


def simplexalgo(A,b,c):
	x,basis_index,feasible,tab,A,b = phase_one(A,b,c)
	if feasible:
		tab,bounded,basis_index = phase_two(x,basis_index,A,b,c)
	else:
		bounded = 0
	return tab,bounded,feasible,basis_index


def lexcolumn(l,tab):
	m,n = tab.shape
	tab = tab[:,1:n]
	indexset = np.where(tab[l+1,:]<0)
	indexset = indexset[0]
	if len(indexset)==0:
		j = []
		return j
	columns = tab[:,indexset]
	for i in range(len(indexset)):
		columns[:,i] = columns[:,i]/columns[l+1,i]
	findlargestset = np.where(columns[0,:]==np.max(columns[0,:]))
	findlargest = findlargestset[0]
	counter = 1
	while len(findlargest)>=2:
		ntem,mtem= columns.shape
		columns = columns[1:ntem,findlargest]
		indexset = indexset[findlargest]
		findlargestset = np.where(columns[0,:]==np.max(columns[0,:]))
		findlargest = findlargestset[0]
		counter += 1
		if counter > m - 1:
			print('counter exceeds max num')
			sys.exit()
	j = indexset[findlargest]
	return j
	

def dualsimplexalgo(tab,basis_index):
	###
	# follow standard layout [obj,cbar;b,A]
	###
	m,n = tab.shape
	cc = 0
	var_selection_order = []
	# modfy the tab to be column lex positive
	for j in range(1,n):
		if (j-1) not in basis_index:
			columnfirstnonzeroset = np.where(abs(tab[0:m,j])>0)
			columnfirstnonzero = columnfirstnonzeroset[0][0]
			if tab[columnfirstnonzero,j]<0:
				if columnfirstnonzero==0:
					raise ValueError('reduced cost is negative for non bfs variable')
					#exit.sys()
				tab[1:m,j] = -tab[1:m,j]
	# start iteration
	while np.sum(tab[1:m,0]<-1e-10)>=1: # or <0
		#print 'iteration...'
		# choose the lth row with x_b[l]<0
		lset = np.where(tab[1:m,0]<0)
		l = lset[0][0]
		# check if lth rows are all positive
		if np.sum(tab[l+1,1:n]>=0)==n-1:
			print('primal nonfeasible due to dual unbounded...')
			# dual unbounded, primal non feasible
			print(tab[l+1,:])
			bounded = 1
			feasible = 0
			tab = []; basis_index = []
			return tab,bounded,feasible,basis_index
		cbar = tab[0,1:n]
		if np.sum(cbar<0)>=1:
			raise ValueError('error due to negative cbar in dual simplex')
			#sys.exit()
		# choose the pivot column
		j = lexcolumn(l,tab)
		# update basis index
		basis_index[l] = j
		# update tab
		for i in range(m):
			if i!=l+1:
				tab[i,:] = tab[i,:] + (-tab[i,j+1]/(0.0+tab[l+1,j+1])) * tab[l+1,:]
				tab[i,j+1] = 0
		tab[l+1,:] = tab[l+1,:]/(0.0+tab[l+1,j+1])
		cc += 1
		var_selection_order.append(l)
		if cc>1000:
			print('dual simplex iteration exceeds max iter')
			print(var_selection_order)
			sys.exit()
	bounded = 1
	feasible = 1
	return tab,bounded,feasible,basis_index


def compute_solution_from_tab(tab,basis_index):
	m,n = tab.shape
	x = np.zeros(n-1)
	basis_index = list(basis_index)
	for i in range(len(basis_index)):
		x[basis_index[i]] = tab[i+1,0]
	return x


def SolveLP(A,b,c):
	c=-c
	tab,bounded,feasible,basis_index = simplexalgo(A,b,c)
	obj = -tab[0,0]
	solution = compute_solution_from_tab(tab,basis_index)
	RC = tab[0,1:]
	return obj,solution,basis_index,RC


def computeoptimaltab(A,b,RC,obj,basis_index):
	'''
	A - A matrix, b - constraint, RC - reduced cost, basis_index - basis 
	'''
	m,n = A.shape
	assert m == b.size; assert n == RC.size
	B = A[:,basis_index]
	try:
		INV = np.linalg.inv(B)
	except:
		print('basisindex length:', basis_index.size)
		print('Ashape:', A.shape)
		raise ValueError
	x = np.dot(INV,b)
	A_ = np.dot(INV,A)
	firstrow = np.append(-obj,RC)
	secondrow = np.column_stack((x,A_))
	tab = np.vstack((firstrow,secondrow))
	return tab


def SolveLPtab(tab,c):
	# extract data from the tab
	m,n = tab.shape
	#print c.size,n
	try:
		assert c.size == n-1
	except:
		print(c.size,tab.shape)
		raise ValueError
	A = tab[1:m,1:n]; b = tab[1:n,0]
	obj,sol,basis,rc = SolveLP(A,b,c) # dual simplex 1
	return obj,sol,basis,rc	


def SolveLPtabDual(tab,c,basis_index):
	# construct the entire tab
	c = -c
	#x = tab[:,0]
	#obj = -np.sum(c[basis_index] * x)
	#bigtab = np.append(obj,c)
	#bigtab = np.vstack((bigtab,tab))
	bigtab = tab
	tab,bounded,feasible,basis_index = dualsimplexalgo(bigtab,basis_index)
	return tab,basis_index


def generatecutzeroth(row):
	###
	# generate cut that includes cost/obj row as well
	###
	n = row.size
	a = row[1:n]
	b = row[0]
	cut_a = a - np.floor(a)
	cut_b = b - np.floor(b)
	return cut_a,cut_b


def generatecut_MIP(row,I,basis_index):
	'''
	generate cut for MIP
	I: set of vars required to be integers
	'''
	n = row.size
	b = row[0]
	a = row[1:n]
	f = a - np.floor(a)
	f0 = b - np.floor(b)
	cut_a = np.zeros(n-1)
	cut_b = 0
	for i in range(n-1):
		if i not in basis_index:
			if i in I:
				if f[i]<=f0:
					cut_a[i] = f[i]/(f0+0.0)
				else:
					cut_a[i] = (1-f[i])/(1+0.0-f0)
			else:
				if a[i]>=0:
					cut_a[i] = a[i]/(f0+0.0)
				else:
					cut_a[i] = -a[i]/(1+0.0-f0)
	cut_b = 1
	return cut_a,cut_b	


def updatetab(tab,cut_a,cut_b,basis_index):
	cut_a = -cut_a
	cut_b = -cut_b
	m,n = tab.shape
	A_ = tab[1:m,1:n]; b_ = tab[1:m,0]; c_ = tab[0,1:n]; obj = tab[0,0]
	Anew1 = np.column_stack((A_,np.zeros(m-1)))
	Anew2 = np.append(cut_a,1)
	Anew = np.vstack((Anew1,Anew2))
	bnew = np.append(b_,cut_b)
	cnew = np.append(c_,0)
	M1 = np.append(obj,cnew)
	M2 = np.column_stack((bnew,Anew))
	newtab = np.vstack((M1,M2))
	basis_index = np.append(basis_index,n-1)
	return newtab,basis_index,Anew,bnew


def PRUNEtab(tab,basis_index,numvar):
	'''
	prune and return a basis_index cleared of redundant slacks
	'''
	aa = np.asarray(basis_index)
	while np.sum(aa>=numvar)>=1:
		tab,basis_index = prunetab(tab,basis_index,numvar)
		aa = np.asarray(basis_index)
	return tab,basis_index


def prunetab(tab,basis_index,numvar):
	'''
	m,n original size of the tab, m: original num of constraints, n: original num of vars (not including slack vars)
	drop the slack variables that enter basis
	'''
	M,N = tab.shape 
	for i in basis_index:
		if i>=numvar:
            # found a slack variable that enters the basis
            # drop the column
			lset = np.where(abs(tab[1:M,i+1]-1)<1e-8)
			l = lset[0][0]
			tab = np.delete(tab,i+1,1)
			tab = np.delete(tab,l+1,0)
			basis_index = list(basis_index)
			basis_index.remove(i)
			for j in range(len(basis_index)):
				if basis_index[j]>i:
					basis_index[j] -= 1
			basis_index = np.asarray(basis_index)
           # print 'pruning...'
			return tab,basis_index
	return tab,basis_index

In [ ]:
import numpy as np

import time


SOLVER = 'GUROBI'



def computeoptimaltab(A, b, RC, obj, basis_index):
	m,n = A.shape
	assert m == b.size; assert n == RC.size
	B = A[:,basis_index]
	try:
		INV = np.linalg.inv(B)
	except:
		print('basisindex length:', basis_index.size)
		print('Ashape:', A.shape)
		raise ValueError
	x = np.dot(INV,b)
	A_ = np.dot(INV,A)
	firstrow = np.append(-obj,RC)
	secondrow = np.column_stack((x,A_))
	tab = np.vstack((firstrow,secondrow))
	return tab


def compute_state(A,b,c):
	m,n = A.shape
	assert m == b.size and n == c.size
	A_tilde = np.column_stack((A,np.eye(m)))
	b_tilde = b
	c_tilde = np.append(c,np.zeros(m))
	if SOLVER == 'GUROBI':
		obj,sol,basis_index,rc = GurobiSolve(A_tilde,b_tilde,c_tilde)
	elif SOLVER == 'SCIPY':
		obj,sol,basis_index,rc = ScipyLinProgSolve(A_tilde,b_tilde,c_tilde)
	tab = computeoptimaltab(A_tilde,b_tilde,rc,obj,basis_index)
	tab = roundmarrays(tab)
	x = tab[:,0]
	#print tab
	done = True
	if np.sum(abs(np.round(x)-x)>1e-2) >= 1:
		done = False
	cuts_a = []
	cuts_b = []
	for i in range(x.size):
		if abs(round(x[i])-x[i])>1e-2: 
			# fractional rows used to compute cut
			cut_a,cut_b = generatecutzeroth(tab[i,:])	
			# a^T x + e^T y >= d
			assert cut_a.size == m+n
			a = cut_a[0:n]
			e = cut_a[n:]
			newA = np.dot(A.T,e) - a
			newb = np.dot(e,b) - cut_b
			cuts_a.append(newA)
			cuts_b.append(newb)
	cuts_a,cuts_b = np.array(cuts_a),np.array(cuts_b)
	return A,b,cuts_a,cuts_b,done,obj,x,tab


def roundmarrays(x,delta=1e-7):
	'''
	if certain components of x are very close to integers, round them
	'''
	index = np.where(abs(np.round(x)-x)<delta)
	x[index] = np.round(x)[index]
	return x


class GurobiOriginalEnv(object):
	def __init__(self, A, b, c, solution=None, reward_type='simple'):
		'''
		min c^T x, Ax <= b, x>=0
		'''
		self.A0 = A.copy()
		self.A = A.copy()
		self.b0 = b.copy()
		self.b = b.copy()
		self.c0 = c.copy()
		self.c = c.copy()
		self.x = None
		self.reward_type = reward_type
		assert reward_type in ['simple', 'obj']

		# upon init, check if the ip problem can be solved by lp
		#try:
		#	_, done = self._reset()
		#	assert done is False
		#except NotImplementedError:
		#	print('the env needs to be initialized with nontrivial ip')
		
	def check_init(self):
		_, done = self._reset()
		return done

	def _reset(self):
		self.A,self.b,self.cuts_a,self.cuts_b,self.done,self.oldobj,self.x,self.tab = compute_state(self.A0,self.b0,self.c0)
		return (self.A,self.b,self.c0,self.cuts_a,self.cuts_b), self.done

	def reset(self):
		s, d = self._reset()
		return s

	def step(self, action):
		cut_a,cut_b = self.cuts_a[action,:],self.cuts_b[action]
		self.A = np.vstack((self.A,cut_a))
		self.b = np.append(self.b,cut_b)
		try:
			self.A,self.b,self.cuts_a,self.cuts_b,self.done,self.newobj,self.x,self.tab = compute_state(self.A,self.b,self.c0)
			if self.reward_type == 'simple':
				reward = -1.0
			elif self.reward_type == 'obj':
				reward = np.abs(self.oldobj - self.newobj)
		except:
			print('error in lp iteration')
			self.done = True
			reward = 0.0
		self.oldobj = self.newobj
		self.A,self.b,self.cuts_a,self.cuts_b = map(roundmarrays,[self.A,self.b,self.cuts_a,self.cuts_b])
		return 	(self.A,self.b,self.c0,self.cuts_a,self.cuts_b), reward, self.done, {}

	def max_gap(self):
		"""
		this method computes the max achivable gap
		"""
		# preprocessing
		A, b, c = self.A0.copy(), self.b0.copy(), self.c0.copy()
		m,n = A.shape
		assert m == b.size and n == c.size
		A_tilde = np.column_stack((A,np.eye(m)))
		b_tilde = b
		c_tilde = np.append(c,np.zeros(m))
		A, b, c = A_tilde, b_tilde, c_tilde
		# compute gaps
		objint, solution_int = gurobiutils.GurobiIntSolve(A, b, c)
		objlp, solution_lp, _, _ = gurobiutils.GurobiSolve(A, b, c)
		return np.abs(objint - objlp), solution_int, solution_lp


class MultipleEnvs(object):
	def __init__(self, envs):
		self.envs = envs
		self.all_indices = list(range(len(self.envs)))
		self.available_indices = list(range(len(self.envs)))
		self.env_index = None
		self.env_now = None

	def reset(self):
		self.env_index = np.random.choice(self.available_indices)
		self.available_indices.remove(self.env_index)
		if len(self.available_indices) == 0:
			self.available_indices = self.all_indices[:]

		self.env_now = self.envs[self.env_index]
		return self.env_now.reset()

	def step(self, action):
		assert self.env_now is not None
		return self.env_now.step(action)


class timelimit_wrapper(object):
	def __init__(self, env, timelimit):
		self.env = env
		self.timelimit = timelimit
		self.counter = 0

	def reset(self):
		self.counter = 0
		return self.env.reset()

	def step(self, action):
		self.counter += 1
		obs, reward, done, info = self.env.step(action)
		if self.counter >= self.timelimit:
			done = True
		return obs, reward, done, info


# some functions for loading instances
def make_multiple_env(load_dir, idx_list, timelimit, reward_type):
	envs = []
	for idx in idx_list:
		print('loading training instances, dir {} idx {}'.format(load_dir, idx))
		A = np.load('{}/A_{}.npy'.format(load_dir, idx))
		b = np.load('{}/b_{}.npy'.format(load_dir, idx))
		c = np.load('{}/c_{}.npy'.format(load_dir, idx))
		env = timelimit_wrapper(GurobiOriginalEnv(A, b, c, solution=None, reward_type=reward_type), timelimit)
		envs.append(env)
	env_final = MultipleEnvs(envs)
	return env_final



In [ ]:

!pip install wandb -qqq
import wandb
wandb.login()
wandb.init()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
### TRAINING

# Setup: You may generate your own instances on which you train the cutting agent.
custom_config = {
    "load_dir"        : 'instances/randomip_n60_m60',   # this is the location of the randomly generated instances (you may specify a different directory)
    "idx_list"        : list(range(20)),                # take the first 20 instances from the directory
    "timelimit"       : 50,                             # the maximum horizon length is 50
    "reward_type"     : 'obj'                           # DO NOT CHANGE reward_type
}

# Easy Setup: Use the following environment settings. We will evaluate your agent with the same easy config below:
easy_config = {
    "load_dir"        : 'instances/train_10_n60_m60',
    "idx_list"        : list(range(10)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}

# Hard Setup: Use the following environment settings. We will evaluate your agent with the same hard config below:
hard_config = {
    "load_dir"        : 'instances/train_100_n60_m60',
    "idx_list"        : list(range(99)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}


test_config = {
    "load_dir" : 'instances/test_100_n60_m60',
    "idx_list" : list(range(99)),
    "timelimit" : 50,
    "reward_type" : 'obj'
}


class LSTM_net(nn.Module):
    def __init__(self, input_size, hidden_size, bidirectional=False):
        super(LSTM_net, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.bidirectional = bidirectional

        self.lstm = nn.LSTM(input_size, hidden_size,
                            bidirectional=bidirectional, batch_first=True)

    def forward(self, input):
        hidden = self.init_hidden()
        inputs = torch.FloatTensor(input).view(1, -1, self.input_size)
        output, _ = self.lstm(inputs)
        # output[-1] is same as last hidden state
        output = output[-1].reshape(-1, self.hidden_size)
        return output

    def init_hidden(self):
        return (torch.zeros(1 + int(self.bidirectional), 1, self.hidden_size),
                torch.zeros(1 + int(self.bidirectional), 1, self.hidden_size))


class Attention_Net(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2):
        super(Attention_Net, self).__init__()
        # constrain and cuts dimension
        self.input_size = int(input_size)
        self.hidden_size = int(hidden_size)
        self.hidden_size2 = int(hidden_size2)
        self.lstm1 = LSTM_net(input_size, hidden_size)
        self.lstm2 = LSTM_net(input_size, hidden_size)
        self.linear1 = nn.Linear(self.hidden_size, self.hidden_size2)
        self.linear2 = nn.Linear(self.hidden_size2, self.hidden_size2)
        self.tanh = nn.Tanh()

    def forward(self, constraints, cuts):
        constraints = torch.FloatTensor(constraints)
        cuts = torch.FloatTensor(cuts)

        # lstm
        A_embed = self.lstm1.forward(constraints)
        D_embed = self.lstm2.forward(cuts)

        # dense
        A = self.linear2(self.tanh(self.linear1(A_embed)))
        D = self.linear2(self.tanh(self.linear1(D_embed)))

        # attention
        logits = torch.sum(torch.mm(D, A.T), axis=1)

        return logits


# Policy network will just be copied from lab4 and make small modification
class Policy(object):
    def __init__(self, input_size, hidden_size, hidden_size2, lr):

        self.model = Attention_Net(input_size, hidden_size, hidden_size2)
        # DEFINE THE OPTIMIZER
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

    def compute_prob(self, constraints, cuts):
        constraints = torch.FloatTensor(constraints)
        cuts = torch.FloatTensor(cuts)
        prob = torch.nn.functional.softmax(self.model(constraints, cuts), dim=-1)
        return prob.cpu().data.numpy()

    def _to_one_hot(self, y, num_classes):
        """
        convert an integer vector y into one-hot representation
        """
        scatter_dim = len(y.size())
        y_tensor = y.view(*y.size(), -1)
        zeros = torch.zeros(*y.size(), num_classes, dtype=y.dtype)
        return zeros.scatter(scatter_dim, y_tensor, 1)

    def train(self, constraints, cuts, actions, Qs):
        """
        states: numpy array (states)
        actions: numpy array (actions)
        Qs: numpy array (Q values)
        """
        actions = torch.LongTensor(actions)
        Qs = torch.FloatTensor(Qs)

        total_loss = 0
        # for a bunch of constraints and cuts, need to go one by one
        for i in range(len(constraints)):
            curr_constraints = constraints[i]
            curr_cuts = cuts[i]
            curr_action = actions[i]
            # COMPUTE probability vector pi(s) for all s in states
            logits = self.model(curr_constraints, curr_cuts)
            prob = torch.nn.functional.softmax(logits, dim=-1)
            # Compute probaility pi(s,a) for all s,a
            action_onehot = self._to_one_hot(curr_action, curr_cuts.shape[0])
            prob_selected = torch.sum(prob * action_onehot, axis=-1)

            # FOR ROBUSTNESS
            prob_selected += 1e-8
            loss = -torch.mean(Qs[i] * torch.log(prob_selected))
            # BACKWARD PASS
            self.optimizer.zero_grad()
            loss.backward()
            # UPDATE
            self.optimizer.step()
            total_loss += loss.detach().cpu().data.numpy()

        return total_loss


def discounted_rewards(r, gamma):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_sum = 0
    for i in reversed(range(0,len(r))):
        discounted_r[i] = running_sum * gamma + r[i]
        running_sum = discounted_r[i]
    return list(discounted_r)


def normalization(A, b, E, d):
    all_coeff = np.concatenate((A, E), axis=0)
    all_constraint = np.concatenate((b, d))
    max_1, max_2 = np.max(all_coeff), np.max(all_constraint)
    min_1, min_2 = np.min(all_coeff), np.min(all_constraint)
    norm_A = (A - min_1) / (max_1 - min_1)
    norm_E = (E - min_1) / (max_1 - min_1)
    norm_b = (b - min_2) / (max_2 - min_2)

    norm_d = (d - min_2) / (max_2 - min_2)

    return norm_A, norm_b, norm_E, norm_d


if __name__ == "__main__":

    training = False
    explore = True
    PATH = "models/easy_config_best_model_2.pt"


    # create env
    env = make_multiple_env(**test_config)
    lr = 1e-2
    # initialize networks
    input_dim = 61
    lstm_hidden = 10
    dense_hidden = 64

    explore_rate = 1.0
    min_explore_rate = 0.01
    max_explore_rate = 0.5
    explore_decay_rate = 0.01
    best_rew = 0

    if training:
        actor = Policy(input_size=input_dim, hidden_size=lstm_hidden, hidden_size2=dense_hidden, lr=lr)
    else:
        actor = torch.load(PATH)

    sigma = 0.2
    gamma = 0.99 # discount
    rrecord = []
    for e in range(50):
        # gym loop
        # To keep a record of states actions and reward for each episode
        obss_constraint = []  # states
        obss_cuts = []
        acts = []
        rews = []

        s = env.reset()   # samples a random instance every time env.reset() is called
        d = False
        repisode = 0

        while not d:

            A, b, c0, cuts_a, cuts_b = s

            # normalization
            A, b, cuts_a, cuts_b = normalization(A, b, cuts_a, cuts_b)

            # concatenate [a, b] [e, d]
            curr_constraints = np.concatenate((A, b[:, None]), axis=1)
            available_cuts = np.concatenate((cuts_a, cuts_b[:, None]), axis=1)

            # compute probability distribution
            prob = actor.compute_prob(curr_constraints, available_cuts)
            prob /= np.sum(prob)

            explore_rate = min_explore_rate + \
                           (max_explore_rate - min_explore_rate) * np.exp(-explore_decay_rate * (e))

            # epsilon greedy for exploration
            if training and explore:
                random_num = random.uniform(0, 1)
                if random_num <= explore_rate:
                    a = np.random.randint(0, s[-1].size, 1)
                else:
                    #a = np.argmax(prob)
                    a = [np.random.choice(s[-1].size,  p=prob.flatten())]
            else:
                # for testing case, only sample action
                a = [np.random.choice(s[-1].size,  p=prob.flatten())]

            new_state, r, d, _ = env.step(list(a))
            #print('episode', e, 'step', t, 'reward', r, 'action space size', new_state[-1].size, 'action', a)
            #a = np.random.randint(0, s[-1].size, 1) # s[-1].size shows the number of actions, i.e., cuts available at state s
            #A, b, c0, cuts_a, cuts_b = new_state

            obss_constraint.append(curr_constraints)
            obss_cuts.append(available_cuts)
            acts.append(a)
            rews.append(r)
            s = new_state
            repisode += r

        # record rewards and print out to track performance
        rrecord.append(np.sum(rews))
        returns = discounted_rewards(rews, gamma)
        # we only use one trajectory so only one-variate gaussian used here.
        Js = returns + np.random.normal(0, 1, len(returns)) / sigma
        print("episode: ", e)
        print("sum reward: ", repisode)

        # PG update and save best model so far
        if training:
            if repisode >= best_rew:
                best_rew = repisode
                torch.save(actor, PATH)

            loss = actor.train(obss_constraint, obss_cuts, acts, Js)
            print("Loss: ", loss)


        #wandb logging
        wandb.log({"Discounted Reward": np.sum(returns)})
        fixedWindow = 10
        movingAverage = 0
        if len(rrecord) >= fixedWindow:
            movingAverage = np.mean(rrecord[len(rrecord) - fixedWindow:len(rrecord) - 1])
        wandb.log({"Training reward": repisode, "training reward moving average": movingAverage})


	#if using hard-config make sure to use "training-hard" tag in wandb.init in the initialization on top

loading training instances, dir instances/test_100_n60_m60 idx 0


FileNotFoundError: ignored